In [1]:
import keras
import pandas as pd
import numpy as np
import matchzoo as mz

Using TensorFlow backend.


## Read Data 

In [2]:
def read_data(path, include_label):
    def scan_file():
        with open(path) as in_file:
            next(in_file)  # skip header
            for l in in_file:
                yield l.strip().split('\t')
    if include_label:
        return [(qid, did, q, d, float(label)) for qid, q, _, _, did, d, label in scan_file()]
    else:
        return [(qid, did, q, d) for qid, q, _, _, did, d, _ in scan_file()]

train = read_data('data/WikiQA/WikiQACorpus/WikiQA-train.tsv', include_label=True)
predict  = read_data('data/WikiQA/WikiQACorpus/WikiQA-test.tsv', include_label=False)

## Preprocess Data

In [3]:
train_pack = mz.pack(train[:1000])
predict_pack = mz.pack(predict[:1000])

In [4]:
preprocessor = mz.preprocessors.DSSMPreprocessor()
preprocessor.fit(train_pack)

Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit: 100%|██████████| 96/96 [00:00<00:00, 3622.12it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit: 100%|██████████| 1000/1000 [00:00<00:00, 3989.77it/s]
Fitting vocabulary unit.: 100%|██████████| 84935/84935 [00:00<00:00, 2678903.08it/s]


In [5]:
train_pack_processed = preprocessor.transform(train_pack)
predict_pack_processed = preprocessor.transform(predict_pack)

Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit => WordHashingUnit: 100%|██████████| 96/96 [00:00<00:00, 4320.64it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit => WordHashingUnit: 100%|██████████| 1000/1000 [00:00<00:00, 3157.26it/s]
Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit => WordHashingUnit: 100%|██████████| 108/108 [00:00<00:00, 5879.10it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit => WordHashingUnit: 100%|██████████| 992/992 [00:00<00:00, 3328.73it/s]


## Generate Data

In [6]:
train_generator = mz.DataGenerator(train_pack_processed, batch_size=32)

## Model Building

In [7]:
ranking_task = mz.tasks.Ranking()
ranking_task.metrics = [
    'mae', 'map', 'precision',
    mz.metrics.Precision(k=3),
    mz.metrics.DiscountedCumulativeGain(k=1),
    mz.metrics.DiscountedCumulativeGain(k=3),
    mz.metrics.DiscountedCumulativeGain(k=5),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=1),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=3),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=5)
]
ranking_task.metrics

['mae',
 mean_average_precision(0),
 precision@1(0),
 precision@3(0),
 discounted_cumulative_gain@1(0),
 discounted_cumulative_gain@3(0),
 discounted_cumulative_gain@5(0),
 normalized_discounted_cumulative_gain@1(0),
 normalized_discounted_cumulative_gain@3(0),
 normalized_discounted_cumulative_gain@5(0)]

In [8]:
model = mz.models.DSSMModel()
model.params['task'] = ranking_task
model.params['input_shapes'] = preprocessor.context['input_shapes']
model.guess_and_fill_missing_params()
model.build()
model.compile()

Parameter "name" set to DSSMModel.


## Model Training

In [9]:
model.fit(*train_pack_processed.unpack(), epochs=2, batch_size=32)

Epoch 1/2
1000/1000 [==============================] - 1s 699us/step - loss: 0.1776 - mean_absolute_error: 0.3276
Epoch 2/2
1000/1000 [==============================] - 0s 419us/step - loss: 0.1659 - mean_absolute_error: 0.3271


In [10]:
model.fit_generator(train_generator, epochs=2, workers=4, use_multiprocessing=True)

Epoch 1/2
32/32 [==============================] - 1s 19ms/step - loss: 0.0602 - mean_absolute_error: 0.1822
Epoch 2/2
32/32 [==============================] - 1s 20ms/step - loss: 0.0255 - mean_absolute_error: 0.1177


### Training with Dynamic Data Generator

In [11]:
pp_no_hashing = mz.preprocessors.DSSMPreprocessor(with_word_hashing=False)
train_pack_processed_no_hashing = pp_no_hashing.fit_transform(train_pack)
word_hashing_unit = mz.processor_units.WordHashingUnit(pp_no_hashing.context['vocab_unit'].state['term_index'])
dynamic_data_generator = mz.data_generator.DynamicDataGenerator(word_hashing_unit.transform,
                                                                train_pack_processed_no_hashing)
model.fit_generator(dynamic_data_generator, epochs=2, workers=4, use_multiprocessing=True)

Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit: 100%|██████████| 96/96 [00:00<00:00, 6628.36it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit: 100%|██████████| 1000/1000 [00:00<00:00, 3725.87it/s]
Fitting vocabulary unit.: 100%|██████████| 84935/84935 [00:00<00:00, 2653304.01it/s]
Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit: 100%|██████████| 96/96 [00:00<00:00, 7011.93it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit: 100%|██████████| 1000/1000 [00:00<00:00, 3975.77it/s]

Epoch 1/2


32/32 [==============================] - 1s 21ms/step - loss: 0.0112 - mean_absolute_error: 0.0781
Epoch 2/2
32/32 [==============================] - 1s 24ms/step - loss: 0.0072 - mean_absolute_error: 0.0622


## Model Predicting 

In [12]:
model.predict(predict_pack_processed[:10].unpack()[0])

array([[0.742013  ],
       [0.23119648],
       [0.47507036],
       [0.7948671 ],
       [0.23675187],
       [0.7501831 ],
       [0.9322906 ],
       [0.81654906],
       [0.9279377 ],
       [0.8422093 ]], dtype=float32)

## Model Evaluating

In [13]:
model.evaluate(*train_pack_processed.unpack())

1000/1000 [==============================] - 0s 123us/step


{'loss': 0.005807360898703337,
 'mean_absolute_error': 0.05651593846082687,
 'mean_average_precision(0)': 0.3333333333333333,
 'precision@1(0)': 0.3333333333333333,
 'precision@3(0)': 0.13194444444444445,
 'discounted_cumulative_gain@1(0)': 0.4808983469629878,
 'discounted_cumulative_gain@3(0)': 0.5338530547483661,
 'discounted_cumulative_gain@5(0)': 0.5338530547483661,
 'normalized_discounted_cumulative_gain@1(0)': 0.3333333333333333,
 'normalized_discounted_cumulative_gain@3(0)': 0.3333333333333333,
 'normalized_discounted_cumulative_gain@5(0)': 0.3333333333333333}

## Automation

In [14]:
train_pack.shuffle(inplace=True)

In [15]:
divider = int(len(train_pack) * 0.9)
train_pack_divided = train_pack[:divider]
test_pack_divided = train_pack[divider:]

In [16]:
director = mz.Director()
director.params['models'] = [
    mz.models.DenseBaselineModel(),
    mz.models.DSSMModel(),
    mz.models.ArcIModel(),
    mz.models.KNRMModel(),
    mz.models.ConvKNRMModel(),
    mz.models.DUETModel()
]
director.params['evals_per_model'] = 3
director.params['train_pack'] = train_pack_divided
director.params['test_pack'] = test_pack_divided
director.params['task'] = ranking_task

In [17]:
trials = director.action(verbose=1)

Start tunning model #1 (total: 6).
Model class: <class 'matchzoo.models.dense_baseline_model.DenseBaselineModel'>
Finish tuning model #1 (total: 6)

Start tunning model #2 (total: 6).
Model class: <class 'matchzoo.models.dssm_model.DSSMModel'>
Finish tuning model #2 (total: 6)

Start tunning model #3 (total: 6).
Model class: <class 'matchzoo.models.arci_model.ArcIModel'>
Finish tuning model #3 (total: 6)

Start tunning model #4 (total: 6).
Model class: <class 'matchzoo.models.knrm_model.KNRMModel'>
Finish tuning model #4 (total: 6)

Start tunning model #5 (total: 6).
Model class: <class 'matchzoo.models.conv_knrm_model.ConvKNRMModel'>
Finish tuning model #5 (total: 6)

Start tunning model #6 (total: 6).
Model class: <class 'matchzoo.models.duet_model.DUETModel'>
Finish tuning model #6 (total: 6)



In [18]:
for trial in trials:
    print('Model Name:', trial['model_params']['name'])
    print("Hyper Parameter Sampled:" , trial['sampled_params'])
    print('Loss:', trial['loss'])
    print()

Model Name: DenseBaselineModel
Hyper Parameter Sampled: {'num_dense_layers': 6.0, 'num_dense_units': 661.0}
Loss: 2727.441162109375

Model Name: DenseBaselineModel
Hyper Parameter Sampled: {'num_dense_layers': 2.0, 'num_dense_units': 50.0}
Loss: 671537.1875

Model Name: DenseBaselineModel
Hyper Parameter Sampled: {'num_dense_layers': 3.0, 'num_dense_units': 310.0}
Loss: 21634.552734375

Model Name: DSSMModel
Hyper Parameter Sampled: {'dim_hidden': 35.0}
Loss: 0.21021421253681183

Model Name: DSSMModel
Hyper Parameter Sampled: {'dim_hidden': 92.0}
Loss: 0.141371488571167

Model Name: DSSMModel
Hyper Parameter Sampled: {'dim_hidden': 287.0}
Loss: 0.07071972638368607

Model Name: ArcIModel
Hyper Parameter Sampled: {'dropout_rate': 0.41000000000000003, 'optimizer': 'adam', 'padding': 'valid'}
Loss: 0.05953873693943024

Model Name: ArcIModel
Hyper Parameter Sampled: {'dropout_rate': 0.74, 'optimizer': 'adagrad', 'padding': 'valid'}
Loss: 0.05837153270840645

Model Name: ArcIModel
Hyper Para

## Misc

### Load Embedding Matrix

In [19]:
model = mz.models.ArcIModel()
model.params['task'] = ranking_task
term_index = preprocessor.context['vocab_unit'].state['term_index']
model.params['embedding_input_dim'] = len(term_index) + 1
model.guess_and_fill_missing_params()
model.build()
model.compile()

Parameter "name" set to ArcIModel.
Parameter "input_shapes" set to [(30,), (30,)].
Parameter "embedding_trainable" set to True.


In [20]:
vocab = preprocessor.context['vocab_unit'].state['term_index']
embedding_matrix = np.random.uniform(
    -0.2, 0.2,
    (model.params['embedding_input_dim'], model.params['embedding_output_dim'])
)
model.load_embedding_matrix(embedding_matrix)